In [2]:
import requests, lxml.html
from bs4 import BeautifulSoup
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime
import pickle

This is a selenium scraper which logs on to Goodreads.com, and goes through a list of users. For each user, it scrapes their shelves page, and grabs all the books they've read and rated, and dumps them into a pickle of tuples. When working at full speed, the scraper can get about 20 reviews in 2 seconds.

The headless variant is identical in code, and designed to work on an AWS instance. 

You should add your own goodreads username and password two cells below.

In [4]:
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

chromedriver = "/usr/lib/chromium-browser/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver) #chrome_options=options
driver.get('https://www.goodreads.com/user/sign_in')

In [5]:
password = '' #set username
username = '' #set password


username_form= driver.find_element_by_id("user_email")
username_form.send_keys(username)

username_form= driver.find_element_by_id("user_password")
username_form.send_keys(password)
username_form.send_keys(Keys.RETURN)

In [6]:
#['  Bitten by Witch Fever: Wallpaper & Arsenic in the Victorian Home', 
#'Hawksley, Lucinda', 
#'4.27', 
#'137', 
#'Oct 2016', 
#'really liked it', 
#'0', 
#'1', 
#'Aug 07, 2018', 
#'Aug 07, 2018', 
#'view']

def shelf_scraper(shelf_name, driver):
    books = driver.find_elements_by_class_name('bookalike')
    
    rating_dict = {'did not like it': 1,
          'it was ok': 2,
          'liked it': 3,
          'really liked it': 4,
          'it was amazing': 5,
          '1 of 5 stars': 0}
    
    for book in books:
        lbook = book.text.split('\n')
        title =  lbook[0].strip()
        author = lbook[1].replace('*','').strip()
        avg_rating = lbook[2]
        times_rated = lbook[3]
        rating_text = lbook[5]
        try:
            rating =  rating_dict[rating_text]
        except:
            rating = rating_text
        comments = lbook[6]
        likes = lbook[7]
        date_added = lbook[-2]
        raw = book.get_attribute('innerHTML').strip()
        try:
            isbn_start = raw.find('isbn')
            isbn = raw[isbn_start+70:isbn_start+80]
            soup = BeautifulSoup(raw, 'lxml')
        except:
            isbn = ''
        review_url = soup.find('a', class_='nobreak').get('href') 
        a= (user, shelf_name, review_url, title, author, avg_rating, times_rated, rating, comments, likes, date_added, isbn)
        print(a)
        results.append(a)

In [7]:
#os.chdir(..)
#os.chdir('metis_03-McNulty')
os.getcwd()

'/home/michael/Dropbox/metis_03-McNulty'

In [9]:
df = pd.read_csv('userids.csv', index_col=None, sep ='\t')
df.head()
userids =  list(df.userid.values)

In [11]:
os.chdir('scrapes')
os.getcwd()

'/home/michael/Dropbox/metis_03-McNulty/scrapes'

This cell checks for users that have already been scraped and removes them from the list

In [12]:
jar = []

for file in os.listdir():
    if file.split(' ')[-1]=='shelf.pkl':
        jar.append(int(file.split(' ')[0]))

print(len(userids))
for file in jar:
    try:
        userids.remove(file)
    except:
        pass
print(len(userids))

print(userids[0], userids[-1])
userids = userids[::-1]
print(userids[0], userids[-1])

37786
37786
2508169 294130
294130 2508169


In [13]:
for user in userids:
    try:
        t0 = time.time()
        user= str(user)
        user_url = 'https://www.goodreads.com/review/list/'+user

        driver.get(user_url)

        possibles = driver.find_elements_by_class_name('actionLinkLite')

        #//*[@id="paginatedShelfList"]/div[8]/a[2]
        #//*[@id="shelves"]/div[1]/a[1]
        #//*[@id="shelves"]/div[2]/a[7]

        shelves = []

        for item in possibles:
            try:
                text = item.text.split(' ')[0]
                link = item.get_attribute('href')
                if text == 'Read' or text == 'Currently' or text == 'Want':
                    pass
                else:
                    if 'shelf' in link:
                        print  (link, text)
                        shelves.append((link, text))
            except:
                print(i, 'ERROR')

        results = []
        for shelf in shelves:
            shelf_url = shelf[0]
            shelf_name =  shelf[1]
            driver.get(shelf_url)

            settings = driver.find_element_by_id('shelfSettingsLink')
            settings.click()
            time.sleep(.1)
            list_button = driver.find_element_by_id('listFieldSetLink')
            list_button.click()
            position_button = driver.find_element_by_id('position_field')
            if position_button.is_selected():
                position_button.click()

            while True:
                t1 =time.time()
                time.sleep(.750)
                shelf_scraper(shelf_name, driver)
                print('')
                print (datetime.datetime.now())
                print('%d books %.3f seconds' % (len(results), time.time()-t1), shelf_name)
                print('')
                try:
                    nextpage = driver.find_element_by_class_name('next_page')
                except:
                    break
                if nextpage.get_attribute('href') == None:
                    break
                else:
                    try:
                        nextpage.click()
                    except:
                        break

        filename = user+' shelf.pkl'
        outfile = open(filename,'wb')
        pickle.dump(results,outfile)
        outfile.close()

        print ('%d TOTAL TIME: %.3f' % (len(results), time.time()-t0))
    except:
        print('ERROR', user)
        pass

https://www.goodreads.com/review/list/294130-manderson?shelf=autobiography autobiography
https://www.goodreads.com/review/list/294130-manderson?shelf=biography biography
https://www.goodreads.com/review/list/294130-manderson?shelf=childrensbooks childrensbooks
https://www.goodreads.com/review/list/294130-manderson?shelf=classicfiction classicfiction
https://www.goodreads.com/review/list/294130-manderson?shelf=contemporaryfiction contemporaryfiction
https://www.goodreads.com/review/list/294130-manderson?shelf=economics economics
https://www.goodreads.com/review/list/294130-manderson?shelf=education education
https://www.goodreads.com/review/list/294130-manderson?shelf=fantasy fantasy
https://www.goodreads.com/review/list/294130-manderson?shelf=graphicnovels graphicnovels
https://www.goodreads.com/review/list/294130-manderson?shelf=history history
https://www.goodreads.com/review/list/294130-manderson?shelf=horror horror
https://www.goodreads.com/review/list/294130-manderson?shelf=manage

ERROR 124117
ERROR 13056902
ERROR 59014
ERROR 30729
ERROR 22979587
ERROR 4253367
ERROR 2689336
ERROR 3398219
ERROR 5898713
ERROR 10044268
ERROR 10945922
ERROR 30516617
ERROR 16175475
ERROR 14089162
ERROR 13363316
ERROR 4055856
ERROR 9827741
ERROR 7118651
ERROR 3724322
ERROR 3282664
ERROR 5024047
ERROR 10910560
ERROR 7173137
ERROR 9363099
ERROR 7335255
ERROR 7328379
ERROR 4356644
ERROR 1331517
ERROR 86417915
ERROR 1435637
ERROR 9886801
ERROR 2275360
ERROR 10788263
ERROR 65401691
ERROR 4746902
ERROR 802619
ERROR 5198425
ERROR 19836892
ERROR 30312601
ERROR 3655826
ERROR 20798930
ERROR 71348511
ERROR 1492212
ERROR 558239
ERROR 1106938
ERROR 1525403
ERROR 37365078
ERROR 959503
ERROR 5202490
ERROR 722450
ERROR 267296
ERROR 17793834
ERROR 585856
ERROR 4378459
ERROR 5135670
ERROR 30357703
ERROR 5134375
ERROR 84789464
ERROR 1971614
ERROR 20474640
ERROR 66672704
ERROR 5362202
ERROR 11375563
ERROR 73933043
ERROR 14720286
ERROR 51684298
ERROR 71791730
ERROR 32200938
ERROR 49619702
ERROR 58354157
E

ERROR 4699735
ERROR 5059523
ERROR 382146
ERROR 1123165
ERROR 29219066
ERROR 5142833
ERROR 7473470
ERROR 35502703
ERROR 26428309
ERROR 3167458
ERROR 3439093
ERROR 5395971
ERROR 6564745
ERROR 3494632
ERROR 2583307
ERROR 1863076
ERROR 11989826
ERROR 5012680
ERROR 10665314
ERROR 9900539
ERROR 4694108
ERROR 18762169
ERROR 1880634
ERROR 45725556
ERROR 44156675
ERROR 1305107
ERROR 4980164
ERROR 2640338
ERROR 51544860
ERROR 12362120
ERROR 8566638
ERROR 5128174
ERROR 13970278
ERROR 20132594
ERROR 2399241
ERROR 27751101
ERROR 6789591
ERROR 35406385
ERROR 8133175
ERROR 8793052
ERROR 10167466
ERROR 16006607
ERROR 7978714
ERROR 7894645
ERROR 8152813
ERROR 18950312
ERROR 4696854
ERROR 8298819
ERROR 5624449
ERROR 8606922
ERROR 4552039
ERROR 37469192
ERROR 71333442
ERROR 3673972
ERROR 20511890
ERROR 4217451
ERROR 2532772
ERROR 28915719
ERROR 1478411
ERROR 625432
ERROR 3289100
ERROR 1666671
ERROR 6092532
ERROR 13656797
ERROR 8098124
ERROR 11881042
ERROR 5267196
ERROR 465249
ERROR 43040300
ERROR 4234595

ERROR 199044
ERROR 23504479
ERROR 8629568
ERROR 41141009
ERROR 7500061
ERROR 1558251
ERROR 14900973
ERROR 6810645
ERROR 19481265
ERROR 50712289
ERROR 45789377
ERROR 68161123
ERROR 31086125
ERROR 36437228
ERROR 5704791
ERROR 285575
ERROR 16397825
ERROR 175709
ERROR 416832
ERROR 10772353
ERROR 2192822
ERROR 11121092
ERROR 7561270
ERROR 30484141
ERROR 939063
ERROR 72731528
ERROR 41513821
ERROR 1206128
ERROR 937382
ERROR 60754399
ERROR 8348
ERROR 20010078
ERROR 9259441
ERROR 22495428
ERROR 3824750
ERROR 4228829
ERROR 288297
ERROR 386833
ERROR 170228
ERROR 225703
ERROR 79923
ERROR 12004101
ERROR 6157592
ERROR 22595720
ERROR 549220
ERROR 67326390
ERROR 1557419
ERROR 1846268
ERROR 1766263
ERROR 18778
ERROR 34364
ERROR 116154
ERROR 146932
ERROR 1429598
ERROR 45331358
ERROR 34371922
ERROR 34953333
ERROR 29863433
ERROR 36451643
ERROR 73572953
ERROR 51942672
ERROR 18006029
ERROR 29698630
ERROR 4507269
ERROR 45738358
ERROR 7262696
ERROR 27077906
ERROR 6557776
ERROR 15044785
ERROR 16429668
ERROR 13

ERROR 30123371
ERROR 19404467
ERROR 39290670
ERROR 12009673
ERROR 23444670
ERROR 7270318
ERROR 32339105
ERROR 27291448
ERROR 11500602
ERROR 20676736
ERROR 1930497
ERROR 35516842
ERROR 21905403
ERROR 8940559
ERROR 48952137
ERROR 67594190
ERROR 63867638
ERROR 71478820
ERROR 54731976
ERROR 75646037
ERROR 14895890
ERROR 22716963
ERROR 53533690
ERROR 63594879
ERROR 4898078
ERROR 64411306
ERROR 62230768
ERROR 77277358
ERROR 6031153
ERROR 193014
ERROR 69625762
ERROR 152503
ERROR 75981
ERROR 4889908
ERROR 756928
ERROR 660104
ERROR 324118
ERROR 5208355
ERROR 4953534
ERROR 8243139
ERROR 5791181
ERROR 16595003
ERROR 15821291
ERROR 146719
ERROR 4577054
ERROR 5290395
ERROR 831094
ERROR 544078
ERROR 5860800
ERROR 237344
ERROR 3920259
ERROR 18084146
ERROR 2027950
ERROR 20944274
ERROR 3571744
ERROR 4016434
ERROR 7036975
ERROR 1101220
ERROR 12991284
ERROR 14943530
ERROR 34353578
ERROR 29685178
ERROR 2376590
ERROR 14510558
ERROR 19893405
ERROR 30714510
ERROR 3600343
ERROR 12235005
ERROR 78792
ERROR 3034

...

ERROR 19313082
ERROR 267156
ERROR 30264891
ERROR 145461
ERROR 1291116
ERROR 1277413
ERROR 1677868
ERROR 6710404
ERROR 157325
ERROR 1964375
ERROR 3803025
ERROR 7319656
ERROR 27846600
ERROR 1779598
ERROR 1627983
ERROR 1008237
ERROR 46150
ERROR 935604
ERROR 9783165
ERROR 34769317
ERROR 17591156
ERROR 18129192
ERROR 4955993
ERROR 4085939
ERROR 62049962
ERROR 6328475
ERROR 5556725
ERROR 8068699
ERROR 70559650
ERROR 616321
ERROR 30635986
ERROR 7452723
ERROR 43223394
ERROR 5933478
ERROR 1359426
ERROR 9346755
ERROR 6817897
ERROR 5197179
ERROR 40025374
ERROR 25243113
ERROR 69521493
ERROR 50057882
ERROR 63444706
ERROR 64803996
ERROR 8128531
ERROR 17124771
ERROR 10907123
ERROR 66549788
ERROR 63991220
ERROR 4426304
ERROR 27747826
ERROR 1265131
ERROR 13138256
ERROR 8913754
ERROR 16747146
ERROR 19594654
ERROR 11180984
ERROR 63584169
ERROR 16146583
ERROR 2532763
ERROR 19971042
ERROR 25792705
ERROR 369876
ERROR 37213849
ERROR 6968514
ERROR 68274023
ERROR 21101224
ERROR 7739343
ERROR 42114800
ERROR 299

ERROR 14454158
ERROR 24011982
ERROR 5346172
ERROR 5070856
ERROR 50041732
ERROR 5749698
ERROR 33943459
ERROR 5839408
ERROR 68610628
ERROR 58033904
ERROR 32330512
ERROR 40365046
ERROR 5595096
ERROR 52955347
ERROR 11351200
ERROR 5712006
ERROR 5591839
ERROR 14247796
ERROR 811952
ERROR 1606895
ERROR 624463
ERROR 116025
ERROR 2736147
ERROR 5606199
ERROR 1271726
ERROR 60538023
ERROR 1410180
ERROR 6695737
ERROR 15528548
ERROR 2275123
ERROR 8809317
ERROR 14450780
ERROR 14877540
ERROR 4444755
ERROR 39587289
ERROR 904335
ERROR 8001403
ERROR 2465701
ERROR 20872231
ERROR 1760824
ERROR 35848351
ERROR 2330106
ERROR 4164661
ERROR 19193279
ERROR 6462583
ERROR 65536298
ERROR 2463977
ERROR 1809046
ERROR 5718497
ERROR 38927308
ERROR 19655198
ERROR 3387843
ERROR 54073230
ERROR 32033450
ERROR 12020580
ERROR 36943042
ERROR 53547296
ERROR 59029422
ERROR 491443
ERROR 11256995
ERROR 11047543
ERROR 26452879
ERROR 4297611
ERROR 5400141
ERROR 2370549
ERROR 27790601
ERROR 42804965
ERROR 626990
ERROR 62590884
ERROR 

ERROR 49072682
ERROR 64129801
ERROR 29220648
ERROR 2744708
ERROR 16245236
ERROR 41317900
ERROR 46089881
ERROR 35980026
ERROR 36348717
ERROR 1768614
ERROR 36269827
ERROR 11828657
ERROR 6313408
ERROR 30847793
ERROR 3444645
ERROR 16978891
ERROR 1234480
ERROR 5016948
ERROR 9542895
ERROR 8003926
ERROR 5534802
ERROR 5312173
ERROR 63515017
ERROR 236894
ERROR 354914
ERROR 16743049
ERROR 150331
ERROR 71996989
ERROR 24965598
ERROR 2418160
ERROR 5058945
ERROR 72176691
ERROR 318724
ERROR 19769021
ERROR 89071
ERROR 14415746
ERROR 76546196
ERROR 1019174
ERROR 3233112
ERROR 5564913
ERROR 3784205
ERROR 27402088
ERROR 60356353
ERROR 69824490
ERROR 37317910
ERROR 27574170
ERROR 25775670
ERROR 49486404
ERROR 1159787
ERROR 40006512
ERROR 69921138
ERROR 30237187
ERROR 10767457
ERROR 32149770
ERROR 7393802
ERROR 1256262
ERROR 18771014
ERROR 61303902
ERROR 14780941
ERROR 30767010
ERROR 74399442
ERROR 803009
ERROR 4126929
ERROR 11204272
ERROR 1820314
ERROR 698727
ERROR 1724590
ERROR 42595590
ERROR 20234777
ER

ERROR 52598215
ERROR 76129
ERROR 59329
ERROR 2499261
ERROR 43342511
ERROR 7407364
ERROR 1039395
ERROR 3672777
ERROR 71848701
ERROR 2213976
ERROR 23580633
ERROR 8161922
ERROR 83144
ERROR 1265504
ERROR 870755
ERROR 82930
ERROR 22106879
ERROR 1302775
ERROR 1023389
ERROR 1310564
ERROR 36644406
ERROR 1740824
ERROR 277741
ERROR 27181674
ERROR 164109
ERROR 5906629
ERROR 4898011
ERROR 3843117
ERROR 314014
ERROR 42681
ERROR 958456
ERROR 70459127
ERROR 6699202
ERROR 121445
ERROR 123494
ERROR 2053594
ERROR 22645716
ERROR 1968826
ERROR 75915014
ERROR 666292
ERROR 1022271
ERROR 1836077
ERROR 1671310
ERROR 69910378
ERROR 1147190
ERROR 666003
ERROR 2028832
ERROR 24778930
ERROR 52263626
ERROR 902083
ERROR 674012
ERROR 643332
ERROR 281406
ERROR 2158493
ERROR 6915263
ERROR 2220787
ERROR 869064
ERROR 958801
ERROR 32913415
ERROR 2531665
ERROR 602210
ERROR 13436114
ERROR 2318873
ERROR 1912701
ERROR 2242482
ERROR 3909756
ERROR 4948093
ERROR 611031
ERROR 1492668
ERROR 603835
ERROR 5742302
ERROR 178320
ERROR 

ERROR 8171548
ERROR 1521805
ERROR 837382
ERROR 15245796
ERROR 14421553
ERROR 2247342
ERROR 13400297
ERROR 31848347
ERROR 8393676
ERROR 3947630
ERROR 7049262
ERROR 4740396
ERROR 39602303
ERROR 325486
ERROR 14413650
ERROR 2079440
ERROR 1589894
ERROR 16294266
ERROR 2984650
ERROR 5368477
ERROR 9120555
ERROR 30177364
ERROR 28875452
ERROR 15897348
ERROR 5082359
ERROR 30460160
ERROR 206177
ERROR 7041883
ERROR 43416074
ERROR 1776786
ERROR 680130
ERROR 38934794
ERROR 671638
ERROR 10118791
ERROR 26193050
ERROR 53213307
ERROR 43250213
ERROR 22872369
ERROR 46410498
ERROR 23650882
ERROR 140286
ERROR 25304792
ERROR 12222751
ERROR 5964785
ERROR 3146558
ERROR 15136285
ERROR 1333081
ERROR 844382
ERROR 40706083
ERROR 1546562
ERROR 17463213
ERROR 5885099
ERROR 1987170
ERROR 4863716
ERROR 6494943
ERROR 40861724
ERROR 799794
ERROR 4697317
ERROR 46271596
ERROR 23841736
ERROR 563794
ERROR 10752470
ERROR 29072829
ERROR 9107978
ERROR 93902
ERROR 12857476
ERROR 22765690
ERROR 3080893
ERROR 21198579
ERROR 175485

A more fully developed version would upload the scrapes to an SQL database rather than flat files.  For a prototype, batch processes were sufficient.

In [ ]:
`